In [39]:
import numpy as np
import pandas as pd

In [40]:
#dataset
from sklearn.datasets import load_iris,fetch_openml

In [41]:
#transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder,MinMaxScaler,RobustScaler

In [42]:
#estimator
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC,LinearSVC

In [43]:
#pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn import set_config

In [44]:
from sklearn.model_selection import train_test_split#データセット分割

In [45]:
#パラメータ調整
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,ParameterGrid

In [46]:
#評価関数
from sklearn.metrics import accuracy_score,f1_score

In [47]:
X,y = load_iris(as_frame=True,return_X_y=True)

In [48]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
dtypes: float64(4)
memory usage: 4.8 KB


In [62]:
#パイプライン構築
##特徴量
features = ['sepal length (cm)',
            'sepal width (cm)',   
            'petal length (cm)', 
            'petal width (cm)']
##変換器パイプライン
###パイプライン定義
transformer = Pipeline(steps=[
    ('scaler',StandardScaler()) #特徴量を標準化
])
###統合（特徴量を変換器にかける）
preprocessor = ColumnTransformer(transformers=[
    ('transform',transformer,features)
])
##パイプライン全体：変換器パイプラインから予測器
pipeline = Pipeline(steps=[
    ('preprocessor',preprocessor),#変換器パイプライン
    ('classifier',LogisticRegression())#予測器（推定器）
])

In [50]:
#パイプライン表示
set_config(display='diagram')
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('transform',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['sepal length (cm)',
                                                   'sepal width (cm)',
                                                   'petal length (cm)',
                                                   'petal width (cm)'])])),
                ('classifier', LogisticRegression())])

In [51]:
set_config(display='None')

In [52]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.5)

In [53]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('transform',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['sepal length (cm)',
                                                   'sepal width (cm)',
                                                   'petal length (cm)',
                                                   'petal width (cm)'])])),
                ('classifier', LogisticRegression())])

In [54]:
y_test_pred = pipeline.predict(X_test)
accuracy_score(y_test,y_test_pred)

0.9333333333333333

In [55]:
#グリッドサーチの設定　
param_grid = [
    {
        'preprocessor__transform__scaler':[StandardScaler(),MinMaxScaler(),RobustScaler()],
        'classifier__C':[0.1,1.0,10.0,100.0],
        'classifier':[LogisticRegression()]
    }
]

grid_search = GridSearchCV(pipeline,param_grid,cv=10,verbose=3,n_jobs=-1)

In [56]:
grid_search.fit(X_train,y_train)

Fitting 10 folds for each of 12 candidates, totalling 120 fits


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('transform',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['sepal '
                                                                          'length '
                                                                          '(cm)',
                                                                          'sepal '
                                                                          'width '
                                                                          '(cm)',
                                                                          'petal '
                                               

In [57]:
print(grid_search.best_params_)#最適なパイプライン

{'classifier': LogisticRegression(C=100.0), 'classifier__C': 100.0, 'preprocessor__transform__scaler': MinMaxScaler()}


In [58]:
print(grid_search.best_score_)#正答率

0.9857142857142858


In [59]:
#テストデータで精度を検証する
y_test_pred = grid_search.predict(X_test)
print('accuracy :',accuracy_score(y_test,y_test_pred))


accuracy : 0.96


In [60]:
#ランダムサーチ
param_grid = [
    {
        "preprocesser__transform__scaler": [StandardScaler(), MinMaxScaler(), RobustScaler()],       
        "classifier__C": [0.1, 1.0, 10.0, 100.0],
        "classifier": [LogisticRegression()]
    },
    {
        "preprocesser__transform__scaler": [StandardScaler(), MinMaxScaler(), RobustScaler()],   
        "classifier__n_estimators": [10, 100, 1000],
        "classifier": [RandomForestClassifier(), GradientBoostingClassifier(), AdaBoostClassifier()]
    },
    {
        "preprocesser__transform__scaler": [StandardScaler(), MinMaxScaler(), RobustScaler()],   
        "classifier__C": [1, 10, 100, 1000],
        "classifier": [SVC(),LinearSVC()]
    }
]

rand_search = RandomizedSearchCV(pipeline, param_grid, cv=10, verbose=3, n_jobs=-1, n_iter=10)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


ValueError: Invalid parameter preprocesser for estimator Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('transform',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['sepal length (cm)',
                                                   'sepal width (cm)',
                                                   'petal length (cm)',
                                                   'petal width (cm)'])])),
                ('classifier', SVC())]). Check the list of available parameters with `estimator.get_params().keys()`.